# Data on Overindebtedness in Germany from Creditreform 

https://www.creditreform.de/aktuelles-wissen/pressemeldungen-fachbeitraege/show/schuldneratlas-deutschland-2018, last retrieved 2019-07-31.



In [46]:
import pandas as pd
import re

def umlaut(string):
    """
    This function replaces replcaces umlaut-code with letter in utf-8. 
    This is useful, when detect function from chardet module does not 
    work.
    dirk.ulbricht@gmail.com, 20.9.2019
    """
    string = re.sub('Ã¼', 'ü',string)
    string = re.sub('ÃŸ', 'ß', string)
    string = re.sub('Ã¶', 'ö', string)
    string = re.sub('Ã¤', 'ä', string)
    return string

file = 'SchuldnerAtlas2018.csv'
creditreform = pd.read_csv(file, sep=';' ,skiprows=8, decimal=',')

# renaming columns, skipping na lines
creditreform.columns = ['nr', 'region', 'state', 'state_abbrev', 'debt_2013', 'debt_2014', 'debt_2015'
                        , 'debt_2016', 'debt_2017', 'debt_2018']
creditreform = creditreform.dropna()

# some state-level information needs to be skipped
creditreform = creditreform[creditreform.state != 'Bundesland']

# Umlaute couldn*t be dealt with using encoding and detect (see functions at top of notebook) therefore:
# correct for wrong reading of German "Umlaute"
creditreform['region'] = creditreform['region'].apply(umlaut)
creditreform['state'] = creditreform['state'].apply(umlaut)

creditreform['debt_2013'] = creditreform['debt_2013'].str.replace(',','.').astype(float)
creditreform['debt_2014'] = creditreform['debt_2014'].str.replace(',','.').astype(float)
creditreform['debt_2015'] = creditreform['debt_2015'].str.replace(',','.').astype(float)
creditreform.shape # There are 401 regions and here are 401 lines

creditreform.set_index('region',inplace=True)
creditreform.rename(index={
    'Aachen, Städteregion': 'Städteregion Aachen',
    'Baden-Baden, Stadtkreis': 'Baden-Baden, Stadt',
    'Düren, Kreis': 'Düren',
    'Freiburg im Breisgau, Stadtkreis': 'Freiburg im Breisgau, Stadt',
    'Heidelberg, Stadtkreis': 'Heidelberg, Stadt',
    'Heilbronn, Stadtkreis': 'Heilbronn, Stadt',
    'Karlsruhe, Stadtkreis': 'Karlsruhe, Stadt',
    'Kleve, Kreis': 'Kleve',
    'Ludwigshafen am Rhein, kreisfr. Stadt': 'Ludwigshafen am Rhein, kreisfreie Stadt',
    'Mannheim, Stadtkreis': 'Mannheim, Universitätsstadt',
    'Mettmann, Kreis': 'Mettmann',
    'Neustadt a.d. Weinstraße, Stadt': 'Neustadt an der Weinstraße, kreisfreie Stadt',
    'Pforzheim, Stadtkreis': 'Pforzheim, Stadt', 
    'Schwabach': 'Schwabach, Stadt',
    'Stuttgart, Stadtkreis': 'Stuttgart, Landeshauptstadt', 
    'Ulm, Stadtkreis': 'Ulm, Universitätsstadt',
    'Viersen, Kreis': 'Viersen',
    'Wesel, Kreis': 'Wesel',
    'Deutschland': 'Bundesgebiet'
},inplace=True)
creditreform.reset_index(inplace=True)

# adding change column
creditreform['ove18_13']=creditreform['debt_2018']-creditreform['debt_2013']
creditreform = creditreform.drop(columns='nr')
creditreform.to_csv('creditreform.csv', index=False)
creditreform.head()

,region,state,state_abbrev,debt_2013,debt_2014,debt_2015,debt_2016,debt_2017,debt_2018,ove18_13
0,Städteregion Aachen,Nordrhein-Westfalen,SV,11.02,11.06,10.94,11.21,11.11,11.18,0.16
1,Ahrweiler,Rheinland-Pfalz,LK,8.72,8.60,8.32,8.61,8.65,8.67,-0.05
2,Aichach-Friedberg,Bayern,LK,5.27,5.19,5.17,5.58,5.62,5.61,0.34
3,Alb-Donau-Kreis,Baden-Württemberg,LK,5.72,5.80,5.90,6.22,6.38,6.49,0.77
4,Altenburger Land,Thüringen,LK,8.29,8.66,8.72,8.84,8.87,8.95,0.66


In [47]:
creditreform_variable_description = pd.DataFrame({
    'Variable' : list(creditreform.columns[creditreform.columns.str.contains('debt')])
    ,'Beschreibung' : """Anteil Überschuldeter Erwachsener; 
                        Für eine genaue Beschreibung der Definition
                        siehe Originalquelle"""
    ,'Description' : """Percentage Share of overindebted adults. For
                        a detailed description of the critria applied
                        for consider adults overindebted see source."""
    ,'Quelle' : 'Creditreform, SchuldnerAtlas 2018'
    ,'Source' : 'Creditreform, SchuldnerAtlas 2018'
})
creditreform_variable_description = creditreform_variable_description.append({'Variable':'ove18_13'
                                         ,'Beschreibung':'Differenz von debt_2018 und debt_2013'
                                         ,'Description':'Difference of debt_2018 and debt_2013'
                                         ,'Source':'Creditreform, SchuldnerAtlas 2018'
                                         ,'Quelle':'Creditreform, SchuldnerAtlas 2018'},ignore_index=True)
creditreform_variable_description = creditreform_variable_description.append({'Variable':'state'
                                         ,'Beschreibung':'Deutsche Ländernamen'
                                         ,'Description':'German state names'
                                         ,'Source':'Creditreform, SchuldnerAtlas 2018'
                                         ,'Quelle':'Creditreform, SchuldnerAtlas 2018'},ignore_index=True)
creditreform_variable_description.to_csv('creditreform_variable_description.csv'
                                         , index=False
                                        ,sep=';')
creditreform_variable_description

,Variable,Beschreibung,Description,Quelle,Source
0,debt_2013,Anteil Überschuldeter Erwachsener; \n ...,Percentage Share of overindebted adults. For\n...,"Creditreform, SchuldnerAtlas 2018","Creditreform, SchuldnerAtlas 2018"
1,debt_2014,Anteil Überschuldeter Erwachsener; \n ...,Percentage Share of overindebted adults. For\n...,"Creditreform, SchuldnerAtlas 2018","Creditreform, SchuldnerAtlas 2018"
2,debt_2015,Anteil Überschuldeter Erwachsener; \n ...,Percentage Share of overindebted adults. For\n...,"Creditreform, SchuldnerAtlas 2018","Creditreform, SchuldnerAtlas 2018"
3,debt_2016,Anteil Überschuldeter Erwachsener; \n ...,Percentage Share of overindebted adults. For\n...,"Creditreform, SchuldnerAtlas 2018","Creditreform, SchuldnerAtlas 2018"
4,debt_2017,Anteil Überschuldeter Erwachsener; \n ...,Percentage Share of overindebted adults. For\n...,"Creditreform, SchuldnerAtlas 2018","Creditreform, SchuldnerAtlas 2018"
5,debt_2018,Anteil Überschuldeter Erwachsener; \n ...,Percentage Share of overindebted adults. For\n...,"Creditreform, SchuldnerAtlas 2018","Creditreform, SchuldnerAtlas 2018"
6,ove18_13,Differenz von debt_2018 und debt_2013,Difference of debt_2018 and debt_2013,"Creditreform, SchuldnerAtlas 2018","Creditreform, SchuldnerAtlas 2018"
7,state,Deutsche Ländernamen,German state names,"Creditreform, SchuldnerAtlas 2018","Creditreform, SchuldnerAtlas 2018"
